In [1]:
import pandas as pd
import numpy as np
import re

from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline,
)
from datasets import load_dataset, Dataset
import torch
import evaluate
import glob

In [2]:
pred_df = pd.read_csv('test.csv')
pred_df

,index,review
0,690427,## Название: 'Война и мир': Грандиозное полотн...
1,253613,"Продукт не следует ни хвалить, ни критиковать,..."
2,379756,'Война и мир' Льва Толстого — это обширный ром...
3,696940,[Вопрос задан во время обсуждения другого вопр...
4,351121,Вот что я придумал: «Война и мир» Льва Толсто...
...,...,...
5995,796464,Лев Толстой. **Объективный отзыв о книге «Вой...
5996,148183,"Указанные темы, такие как «взаимоотношения, др..."
5997,239032,На английском языке Of all the literary works ...
5998,571112,"Тема: Разнотемпераментность персонажей, их хар..."


In [3]:
sample = pd.DataFrame({'index':pred_df['index'], 'sentiment': [0]*len(pred_df)})
sample

,index,sentiment
0,690427,0
1,253613,0
2,379756,0
3,696940,0
4,351121,0
...,...,...
5995,796464,0
5996,148183,0
5997,239032,0
5998,571112,0


In [ ]:
sample.to_csv('')

In [2]:
df = pd.read_csv('train.csv')
df.drop(columns=['index'], inplace=True)
df

,review,sentiment
0,"Есть много причин, по которым 'Война и мир' Ль...",1
1,"Напишите 5 предложений. 1. ""Война и мир"" — это...",2
2,[Практикуйте «Другие люди»] Отзыв о «Войне и ...,1
3,Стремитесь к точному и объективному представле...,2
4,"В книге ""Война и мир"" Льва Толстого разворачив...",2
...,...,...
23995,"Вот и все, что нужно сделать: Во-первых, объяс...",1
23996,"Обязательно укажите, что именно в тексте/сюжет...",0
23997,"Кто, из героев вас заинтересовал и почему? Как...",1
23998,"Жду историй о том, как книга изменила ваше вос...",1


In [3]:
df.isna().sum()

review       0
sentiment    0
dtype: int64

In [4]:
df['review'] = df['review'].apply(lambda x: x.lower())
df

,review,sentiment
0,"есть много причин, по которым 'война и мир' ль...",1
1,"напишите 5 предложений. 1. ""война и мир"" — это...",2
2,[практикуйте «другие люди»] отзыв о «войне и ...,1
3,стремитесь к точному и объективному представле...,2
4,"в книге ""война и мир"" льва толстого разворачив...",2
...,...,...
23995,"вот и все, что нужно сделать: во-первых, объяс...",1
23996,"обязательно укажите, что именно в тексте/сюжет...",0
23997,"кто, из героев вас заинтересовал и почему? как...",1
23998,"жду историй о том, как книга изменила ваше вос...",1


In [5]:
df['review'] = df['review'].apply(lambda x: re.sub(r'\W+', ' ', x))
df

,review,sentiment
0,есть много причин по которым война и мир льва ...,1
1,напишите 5 предложений 1 война и мир это роман...,2
2,практикуйте другие люди отзыв о войне и мире ...,1
3,стремитесь к точному и объективному представле...,2
4,в книге война и мир льва толстого разворачивае...,2
...,...,...
23995,вот и все что нужно сделать во первых объяснит...,1
23996,обязательно укажите что именно в тексте сюжете...,0
23997,кто из героев вас заинтересовал и почему каков...,1
23998,жду историй о том как книга изменила ваше восп...,1


In [6]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

c:\Users\ffedo\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
def process_function(text):
  return tokenizer(
        text['review'],
        truncation=True
  )

In [8]:
dataset = Dataset.from_pandas(df)
dataset = dataset.class_encode_column('sentiment')
dataset = dataset.map(process_function)
dataset = dataset.with_format('torch')
dataset = dataset.train_test_split(test_size=0.2, stratify_by_column='sentiment', seed=1001)
dataset

Stringifying the column:   0%|          | 0/24000 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/24000 [00:00<?, ? examples/s]

Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 19200
    })
    test: Dataset({
        features: ['review', 'sentiment', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4800
    })
})

In [9]:
train_dataset, eval_dataset = dataset['train'], dataset['test']
train_dataset[0]

{'review': 'поясните пожалуйста задачу если это вопрос по теории литературы то вот скажем ответ в одном предложении война и мир роман л н толстого написанный в 1863 1869 гг и являющийся одним из величайших достижений мировой литературы и это был бы правильный ответ если же вы готовите доклад для школы или университета то в зависимости от объёма и цели вашего выступления вам нужно будет рассказать о жанре основных героях и сюжете произведения если же вас просто спросили что такое война и мир и не надо много говорить тогда просто скажите война и мир это книга написанная л н толстым про события 1812 года главный герой романа пьер безухов и наконец если же вопрос поставлен именно так как вы его написали как вы бы рассказали о войне и мире просто без личного отношения то я бы ответил так просто без личного отношения я не смог бы рассказать о войне и мире потому что это величайшее произведение мировой литературы и любое суждение о нём должно основываться на личном отношении к нему а что каса

In [10]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
len(set(df.sentiment))

3

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-uncased",
    num_labels=3
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
training_args = TrainingArguments(
    output_dir = "results",
    learning_rate = 3e-4,
    per_device_train_batch_size = 64,
    per_device_eval_batch_size = 8,
    num_train_epochs = 10,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
    save_total_limit = 3,
    fp16 = True
)

In [18]:
accuracy = evaluate.load("f1")
def compute_metrics(output):
  preds, labels = output
  preds = np.argmax(preds, axis=1) # numpy arrays
  return accuracy.compute(predictions=preds, references=labels)
  # return {"f1": (preds == labels).sum() / len(preds)}

In [19]:
compute_metrics(
    (np.array([
        [
          0.8,
          0.9,
          0.7,
        ],
        [
          0.9,
          0.8,
          0.7,
        ]
]), np.array([1,1]))
)

{'f1': 0.6666666666666666}

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

c:\Users\ffedo\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [17]:
trainer.train()

  0%|          | 0/3000 [00:00<?, ?it/s]

KeyboardInterrupt: 